# SMART FORECASTER

### Product Information:
In the Hospitality Industry the planning of labor is vital. Fulfilling the business needs during high and low times becomes complicated and might vary depending on the particular department. The front office operations strongly depend on scheduling procedures and smart staffing. This includes not only “covering the desk” at all times, but also being efficient, strategic and analytical when it comes to labor costs estimation. 

The main concept of “Smart Forecaster” application is based on the check in amount of guests in the particular date range in the last few years. The application will forecast, according to the historical information, and advise management staff about the amount of workers they will need to schedule at any given time of the day.


* Authors: Kiran Kaushal Kopalley, Mai Phuong Pham , Valeriya Golitsyna, Omar Rahman, Donnie Williams
* Date: 12/03/2019

In [1]:
#This code is used to toggle the code cells in Jupyter Notebook. Just to ensure safety of the code.
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
# importing the csv library
import csv
# importing the getpass library
import getpass
# importing the datetime library with alias as dt
from datetime import datetime as dt
# importing the timedelta with alias as td
from datetime import timedelta as td
# importing the numpy library with alias as np
import numpy as np
# importing the math library
import math
# importing the os library
import os
# importing the pandas library as pd
import pandas as pd
#importing display library
from IPython.display import display
#initialising login to False
login = False

# loop to check the username and password whether the entered keys are correct/not
while login == False:
    # A log in which uses a databases to find multiple usernames and passwords
    data=[]
    with open("Users.csv") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            data.append(row)
    #commented the data variable, uncomment it to get all the username-password combinations which work for this system
    #print(data)
    print("Welcome to SmartForecaster! \n")
    #prompt the user to enter username
    username = input("Please enter username: ")
    #prompt the user to enter password
    password = getpass.getpass("Please enter password: ")
    #loops through all the data in column 0 and 1 assigns it to a variable called 'colA/colB'
    colA = [x[0] for x in data]
    colB = [x[1] for x in data]
    #print(colA)
    #print(ColB)

    if username in colA:

        #iterates through the list for the length of the list
        for k in range (0, len(colA)):
            #checks if username matches the password in the a joining column
            if colA[k] == username and colB[k] == password:
                # Display options once the user logs in
                print("You are logged in.\n")
                print("Enter 1 to Forecast.")
                print("Enter 2 to Update Historical Data.")
                print("Enter 3 to Logout")
                print("")
                login = True
                option_input = int(input())
                # Check which input the user has entered and go to the appropriate field
                if (option_input == 1):
                    print("Welcome to Forecast Page")
                    # Get start date input and validate date format
                    print('Please enter the start date of forecast (YYYY-MM-DD):')
                    # Initialize a value for startDate
                    startDate = 'some date'
                    while not startDate == '':
                        startDate = input()
                        # Validate date format as YYYY-MM-DD
                        try:
                            startDate = dt.strptime(startDate,'%Y-%m-%d')
                            break
                        # Prompt user to input in correct format
                        except ValueError:
                            print('Please enter date in format YYYY-MM-DD:')
                            continue 

                    #Create a list for date
                    dateList = []
                    #Add startDate to list
                    dateList.append(startDate)
                    #Initialize date variable as start date
                    date = startDate
                    #Add 6 more days into list
                    while len(dateList)<7:
                        addDate = date+td(days=1)
                        date=addDate
                        dateList.append(addDate)

                    #Convert List to dictionary and initialize all values to 0
                    dateArrival ={i:0 for i in dateList}
                    #Enter arrival for 7 days as key
                    for i in dateArrival.keys():
                        print('Please enter arrival for the date '+str(i))
                        arrival = int(input())
                        dateArrival[i]=arrival 


                    # Convert dateArrival into a dateframe 
                    arrivalTY = pd.DataFrame(list(dateArrival.items()), columns=['Date', 'New Arrival'])

                    # Break total arrival into different shifts for each day with ratio
                    # 20% bookings check in during day shift, 70% check in during swing and 10% check in during grave
                    arrivalTY = arrivalTY.assign(Day_Arrival = arrivalTY['New Arrival']*0.2)
                    arrivalTY = arrivalTY.assign(Swing_Arrival = arrivalTY['New Arrival']*0.7)
                    arrivalTY = arrivalTY.assign(Grave_Arrival = arrivalTY['New Arrival']*0.1)

                    # Break date into year and date
                    arrivalTY['Year'] = arrivalTY['Date'].dt.year  
                    arrivalTY['Date'] = arrivalTY['Date'].dt.strftime('%m-%d')

                    # Read the excel historical file for the data
                    arrivalLY = pd.read_excel('DATA .xlsx') 
                    # Fill the missing data with 0
                    arrivalLY = arrivalLY.fillna(0)
                    # Convert Date into datetime format
                    arrivalLY['Date'] = pd.to_datetime(arrivalLY['Date'],format='%Y-%m-%d')
                    # Break date into year and date
                    arrivalLY['Year'] = arrivalLY['Date'].dt.year 
                    arrivalLY['Date'] = arrivalLY['Date'].dt.strftime('%m-%d')

                    # Remove outliers
                    # Select data about the same date last year only
                    arrivalLY = arrivalLY[arrivalLY['Date'].isin(list(arrivalTY['Date']))]
                    # To select data of the previous year only
                    # Extract this year forecast into a list
                    yearForecast = list(arrivalTY['Year'])
                    # Subtract 1 from each variable in list
                    compareYear = [x-1 for x in yearForecast]
                    # Select year data that match year data in compareYear from data of last year arrival
                    arrivalLY = arrivalLY[arrivalLY['Year'].isin(compareYear)] 

                    # Merge 2 dataframes of same date this year and last year
                    combinedData = pd.merge(arrivalTY,arrivalLY, on = 'Date', how = 'left')

                    # Calculate the difference of this year data and last year data for each shift
                    combinedData['day Diff'] = combinedData['Day_Arrival'] - combinedData['Arrival of guests during day']
                    combinedData['swing Diff'] = combinedData['Swing_Arrival'] - combinedData['Arrival of guests during swing']
                    combinedData['grave Diff'] = combinedData['Grave_Arrival'] - combinedData['Arrival of guests during grave']
                    # Forecast labor for day shift
                    shift = ['day','swing','grave']
                    for s in range(len(shift)): 
                        combinedData['Expected '+shift[s]+' Staff'] = combinedData['Amount of workers at '+shift[s]]+round((combinedData[shift[s]+' Diff']-100)/50)
                    # Select Columns to print for viewers
                    columnsToSee = ['Date','Total_Arrival','New Arrival','Year_x','Amount of workers at day','Amount of workers at swing','Amount of workers at grave','day Diff','swing Diff','grave Diff','Expected day Staff','Expected swing Staff','Expected grave Staff']
                    combinedData = combinedData.loc[:,columnsToSee]
                    # display the dataframe to user to make changes in the predictions
                    display(combinedData)

                    while True :
                        # Display the options for the user to edit
                        print('Enter an option from the below content ( -1 to exit editing ):')
                        print('1. Modify Day staff')
                        print('2. Modify Swing staff')
                        print('3. Modify Grave staff')
                        inp = int(input())
                        # To edit Day shift
                        if (inp == 1):
                            # Prompt user to enter the index for which the user wants to edit in the Expected Day shift column
                            print('Enter the index of the day which you want to edit:')
                            index = int(input())
                            # Prompt user to enter the value for that index for which the user wants to edit in the Expected Day shift column
                            print('Enter the value:')
                            val = float(input())
                            # set_value function will set the value which the user has entered to the index 
                            combinedData.set_value(index, 'Expected day Staff', val)
                        
                         # To edit swing shift
                        elif (inp == 2):
                            # Prompt user to enter the index for which the user wants to edit in the Expected Swing shift column
                            print('Enter the index of the day which you want to edit:')
                            index = int(input())
                            # Prompt user to enter the value for that index for which the user wants to edit in the Expected Swing shift column
                            print('Enter the value:')
                            val = float(input())
                            # set_value function will set the value which the user has entered to the index
                            combinedData.set_value(index, 'Expected swing Staff', val)

                        # To edit Grave shift    
                        elif (inp == 3):
                            # Prompt user to enter the index for which the user wants to edit in the Expected Grave shift column
                            print('Enter the index of the day which you want to edit:')
                            index = int(input())
                            # Prompt user to enter the value for that index for which the user wants to edit in the Expected Grave shift column
                            print('Enter the value:')
                            val = float(input())
                            # set_value function will set the value which the user has entered to the index
                            combinedData.set_value(index, 'Expected grave Staff', val)
                        # To quit editing
                        elif (inp == -1):
                            break
                        # Warning user to check the input entered
                        else:
                            print('Please check the number and retry')


                    # Hard coding the filename in which the prediction info gets generated, hourlypay and fulltime
                    fileName = 'Prediction Data.xlsx'
                    hourlyPay = 18
                    fullTime = 8
                    # Generate labor expense function
                    def labor_Expense(rate,hour,staff):
                        laborExpense = rate*hour*staff
                        return laborExpense
                    # write the dataframe to excel
                    combinedData.to_excel(fileName, sheet_name='Latest', index = False)
                    # sum of all the employees required for Day shift
                    totalDay = combinedData['Expected day Staff'].sum()
                    # sum of all the employees required for Swing shift
                    totalSwing = combinedData['Expected swing Staff'].sum()
                    # sum of all the employees required for Grave shift
                    totalGrave = combinedData['Expected grave Staff'].sum()
                    # total employee sum
                    totalStaff = totalDay+totalSwing+totalGrave
                    # Displaying total number of employees required for the particular week
                    print('Total employees required for the week '+str(totalStaff))
                    # Displaying total expense for the week
                    print('Labor expenses for the week is estimated at $'+str(labor_Expense(hourlyPay, fullTime, totalStaff)))
                # Update section when user enters option 2
                elif (option_input == 2):
                    # Prompt user for filename
                    fileName = input('Please enter the file name for uploading (include .xlsx file):')

                    # Read new_data excel for new data, set first column Date to be index
                    newData = pd.read_excel(fileName, index_col=[0]) 

                    # Read historical data excel, set first column Date to be index
                    historicalData = pd.read_excel('DATA .xlsx',index_col=[0])

                    # Use reindex and unite the new data to historical data
                    historicalData=historicalData.reindex(historicalData.index.union(newData.index))

                    # In case data are overlap in both dataframes, change all overlap data into missing data
                    historicalData.loc[historicalData.index.intersection(newData.index),:] = np.nan

                    # Fill all missing data in historical dataframe with values in newData dataframe
                    historicalData = historicalData.combine_first(newData)
                    print(historicalData)
                    # Export dataframe back to excel file
                    historicalData.to_excel('DATA .xlsx')
                # Logs out user if he chooses 3 as an option
                else:
                    print('You are successfully logged out.')
                    break
    # Warn the user to check the username and password when they enter wrong
    else:
        print("\n Wrong username or password. Please try again.\n")




Welcome to SmartForecaster! 

